# データセットの改修

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_unimol = pd.read_csv('/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset.csv')
df_unimol

## （blankつける）

In [ ]:
#df_unimolのdelta_valueの絶対値がが0.3~0.5の行を削除
df_filtered = df_unimol[~df_unimol['delta_value'].abs().between(0.3, 0.5)]

df_filtered.to_csv('/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_blank.csv', index=False)

In [ ]:
df_filtered

# FRAGごとのラベリング（そもそも今の評価のコードが単一ラベルになっているか確認）

In [ ]:
import re

def normalize_frag(frag_str):
    """
    フラグメント文字列からダミー原子のインデックスを正規化する
    例: [4*] -> [*], [11*] -> [*], [12*] -> [*]
    """
    if pd.isna(frag_str):
        return frag_str
    # ダミー原子のパターン（[数字*]）を[*]に置換
    normalized = re.sub(r'\[\d+\*\]', '[*]', str(frag_str))
    return normalized

# REF-FRAGとPRB-FRAGを正規化
df_unimol['REF-FRAG_normalized'] = df_unimol['REF-FRAG'].apply(normalize_frag)
df_unimol['PRB-FRAG_normalized'] = df_unimol['PRB-FRAG'].apply(normalize_frag)

# 正規化されたFRAGペアでグループ化してlabel_binの一貫性をチェック
grouped_frag = df_unimol.groupby(['REF-FRAG_normalized', 'PRB-FRAG_normalized'])['label_bin'].nunique().reset_index()
inconsistent_pairs_frag = grouped_frag[grouped_frag['label_bin'] > 1][['REF-FRAG_normalized', 'PRB-FRAG_normalized']]

print(f"非一貫なFRAGペア数: {len(inconsistent_pairs_frag)}")
print("非一貫なFRAGペア:")
print(inconsistent_pairs_frag)

# 非一貫なペアを持つ行を削除
df_consistent_frag = df_unimol.merge(
    inconsistent_pairs_frag, 
    on=['REF-FRAG_normalized', 'PRB-FRAG_normalized'], 
    how='left', 
    indicator=True
)
df_consistent_frag = df_consistent_frag[df_consistent_frag['_merge'] == 'left_only'].drop(columns=['_merge'])

print(f"\n元のデータ行数: {len(df_unimol)}")
print(f"フィルタ後の行数: {len(df_consistent_frag)}")
print(f"削除された行数: {len(df_unimol) - len(df_consistent_frag)}")

# 正規化用の列を削除（必要に応じて）
df_consistent_frag = df_consistent_frag.drop(columns=['REF-FRAG_normalized', 'PRB-FRAG_normalized'])

display(df_consistent_frag.head())

In [ ]:
# 詳細分析: ダミー原子パターンと非一貫性の確認
print("=== ダミー原子パターンの分析 ===")

# ダミー原子のパターンを抽出
dummy_atoms = set()
for frag in df_unimol['REF-FRAG'].dropna():
    matches = re.findall(r'\[\d+\*\]', str(frag))
    dummy_atoms.update(matches)
for frag in df_unimol['PRB-FRAG'].dropna():
    matches = re.findall(r'\[\d+\*\]', str(frag))
    dummy_atoms.update(matches)

print(f"検出されたダミー原子パターン: {sorted(dummy_atoms)}")

# 非一貫なペアの詳細確認
if len(inconsistent_pairs_frag) > 0:
    print(f"\n=== 非一貫なペアの詳細 (最初の5ペア) ===")
    for i, (_, row) in enumerate(inconsistent_pairs_frag.head().iterrows()):
        ref_norm = row['REF-FRAG_normalized']
        prb_norm = row['PRB-FRAG_normalized']
        
        # このペアに対応する元のデータを取得
        mask = (df_unimol['REF-FRAG_normalized'] == ref_norm) & \
               (df_unimol['PRB-FRAG_normalized'] == prb_norm)
        subset = df_unimol[mask][['REF-FRAG', 'PRB-FRAG', 'label_bin', 'delta_value']]
        
        print(f"\n{i+1}. 正規化ペア:")
        print(f"   REF-FRAG_normalized: {ref_norm}")
        print(f"   PRB-FRAG_normalized: {prb_norm}")
        print(f"   このペアの実際のデータ:")
        display(subset)
        
        if i >= 4:  # 最初の5ペアまで
            break

In [ ]:
# 結果をファイルに保存
output_path = '/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentfrag.csv'
df_consistent_frag.to_csv(output_path, index=False)
print(f"フィルタ後のデータを保存しました: {output_path}")

# # 削除されたペアの情報も保存
# inconsistent_info_path = '/gs/bs/tga-ohuelab/masunaga/MMP_dataset/inconsistent_frag_pairs.csv'
# inconsistent_pairs_frag.to_csv(inconsistent_info_path, index=False)
# print(f"非一貫ペアの情報を保存しました: {inconsistent_info_path}")

print(f"\n=== 処理結果サマリー ===")
print(f"元のデータ行数: {len(df_unimol):,}")
print(f"フィルタ後の行数: {len(df_consistent_frag):,}")
print(f"削除された行数: {len(df_unimol) - len(df_consistent_frag):,}")
print(f"削除率: {((len(df_unimol) - len(df_consistent_frag)) / len(df_unimol) * 100):.2f}%")
print(f"非一貫なFRAGペア数: {len(inconsistent_pairs_frag):,}")

In [ ]:
#さらにdf_consistent_fragのdelta_valueの絶対値がが0.3~0.5の行を削除
df_filtered = df_consistent_frag[~df_consistent_frag['delta_value'].abs().between(0.3, 0.5)]
display(df_filtered)
df_filtered.to_csv('/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentfrag+blank.csv', index=False)

In [ ]:
#df_filteredは単一のラベルを持つSMILESペアのみを含むことを確認
grouped_final = df_filtered.groupby(['REF-SMILES', 'PRB-SMILES'])['label_bin'].nunique().reset_index()
inconsistent_final = grouped_final[grouped_final['label_bin'] > 1]
print(f"最終データセット内の非一貫なSMILESペア数: {len(inconsistent_final)}")

# SMILESペアの単一ラベル

In [ ]:
# df_unimolのREF-SMILESとPRB-SMILESの組で、label_binが全て一緒じゃない組を削除
grouped = df_unimol.groupby(['REF-SMILES', 'PRB-SMILES'])['label_bin'].nunique().reset_index()
inconsistent_pairs = grouped[grouped['label_bin'] > 1][['REF-SMILES', 'PRB-SMILES']]
df_consistent_smiles = df_unimol.merge(inconsistent_pairs, on=['REF-SMILES', 'PRB-SMILES'], how='left', indicator=True)
df_consistent_smiles = df_consistent_smiles[df_consistent_smiles['_merge'] == 'left_only'].drop(columns=['_merge'])

print(inconsistent_pairs)
display(df_consistent_smiles)

In [ ]:
inconsistent_pairs_smiles = grouped[grouped['label_bin'] > 1][['REF-SMILES', 'PRB-SMILES']]


In [ ]:
# 結果をファイルに保存
output_path = '/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentsmiles.csv'
df_consistent_smiles.to_csv(output_path, index=False)
print(f"フィルタ後のデータを保存しました: {output_path}")

# # 削除されたペアの情報も保存
# inconsistent_info_path = '/gs/bs/tga-ohuelab/masunaga/MMP_dataset/inconsistent_smiles_pairs.csv'
# inconsistent_pairs_smiles.to_csv(inconsistent_info_path, index=False)
# print(f"非一貫ペアの情報を保存しました: {inconsistent_info_path}")

print(f"\n=== 処理結果サマリー ===")
print(f"元のデータ行数: {len(df_unimol):,}")
print(f"フィルタ後の行数: {len(df_consistent_smiles):,}")
print(f"削除された行数: {len(df_unimol) - len(df_consistent_smiles):,}")
print(f"削除率: {((len(df_unimol) - len(df_consistent_smiles)) / len(df_unimol) * 100):.2f}%")
print(f"非一貫なSMILESペア組数: {len(inconsistent_pairs_smiles):,}")

In [ ]:
#さらにdf_consistent_fragのdelta_valueの絶対値がが0.3~0.5の行を削除
df_filtered = df_consistent_smiles[~df_consistent_smiles['delta_value'].abs().between(0.3, 0.5)]
display(df_filtered)
df_filtered.to_csv('/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentsmiles+blank.csv', index=False)

# SMILESペアの単一ラベル（順不同版）

In [ ]:
# df_unimolのREF-SMILESとPRB-SMILESの組で、label_binが全て一緒じゃない組を削除（順不同で処理）

def create_unordered_pair_key(row):
    """REF-SMILESとPRB-SMILESを辞書順でソートしてペアキーを作成"""
    smiles_pair = tuple(sorted([row['REF-SMILES'], row['PRB-SMILES']]))
    return f"{smiles_pair[0]}||{smiles_pair[1]}"

# 順不同の分子ペアキーを作成
df_unimol['molecule_pair_unordered'] = df_unimol.apply(create_unordered_pair_key, axis=1)

# 各分子ペア（順不同）について、label_binの種類数を計算
grouped_unordered = df_unimol.groupby('molecule_pair_unordered')['label_bin'].nunique().reset_index()
grouped_unordered.columns = ['molecule_pair_unordered', 'label_bin_nunique']

# label_binが複数種類存在するペアを特定
inconsistent_pairs_keys = grouped_unordered[grouped_unordered['label_bin_nunique'] > 1]['molecule_pair_unordered'].tolist()

# 非一貫なペアを除外
df_consistent_smiles_unordered = df_unimol[~df_unimol['molecule_pair_unordered'].isin(inconsistent_pairs_keys)].copy()

# 作業用カラムを削除
df_consistent_smiles_unordered = df_consistent_smiles_unordered.drop(columns=['molecule_pair_unordered'])
df_unimol = df_unimol.drop(columns=['molecule_pair_unordered'])

print(f"非一貫なSMILESペア（順不同）数: {len(inconsistent_pairs_keys)}")
print(f"元のデータ行数: {len(df_unimol):,}")
print(f"フィルタ後の行数: {len(df_consistent_smiles_unordered):,}")
print(f"削除された行数: {len(df_unimol) - len(df_consistent_smiles_unordered):,}")
display(df_consistent_smiles_unordered)

In [ ]:
# 統計情報の確認
inconsistent_pairs_smiles_unordered_count = len(inconsistent_pairs_keys)
print(f"非一貫なSMILESペア（順不同）数: {inconsistent_pairs_smiles_unordered_count}")

In [ ]:
# 結果をファイルに保存
output_path = '/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentsmiles_unordered.csv'
df_consistent_smiles_unordered.to_csv(output_path, index=False)
print(f"フィルタ後のデータを保存しました: {output_path}")

print(f"\n=== 処理結果サマリー ===")
print(f"元のデータ行数: {len(df_unimol):,}")
print(f"フィルタ後の行数: {len(df_consistent_smiles_unordered):,}")
print(f"削除された行数: {len(df_unimol) - len(df_consistent_smiles_unordered):,}")
print(f"削除率: {((len(df_unimol) - len(df_consistent_smiles_unordered)) / len(df_unimol) * 100):.2f}%")
print(f"非一貫なSMILESペア（順不同）数: {inconsistent_pairs_smiles_unordered_count:,}")

In [ ]:
# さらにdf_consistent_smiles_unorderedのdelta_valueの絶対値が0.3~0.5の行を削除
df_filtered_unordered = df_consistent_smiles_unordered[~df_consistent_smiles_unordered['delta_value'].abs().between(0.3, 0.5)]
display(df_filtered_unordered)
    "df_filtered_unordered.to_csv('/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentsmiles_unordered+blank.csv', index=False)"

# df_consistent_smilesのラベル分布の可視化

In [ ]:
# df_consistent_smilesのdelta_value分布の可視化
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df_consistent_smiles = pd.read_csv('/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentsmiles.csv')
plt.figure(figsize=(10, 6))
sns.histplot(df_consistent_smiles['delta_value'], bins=41, kde=False)
plt.title('Distribution of ΔpIC50', fontsize=20)
plt.xlabel('ΔpIC50', fontsize=20)
plt.ylabel('Pairs', fontsize=20)

# 軸の目盛りの文字サイズも20に
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

# 点線を追加（=0, =0.3, =-0.3）
plt.axvline(x=0, color='red', linestyle='--', linewidth=1.5, alpha=1, label='No difference')
plt.axvline(x=0.3, color='orange', linestyle='--', linewidth=1.5, alpha=1, label='Threshold = ±0.3')
plt.axvline(x=-0.3, color='orange', linestyle='--', linewidth=1.5, alpha=1)

# 凡例を左上に追加
plt.legend(loc='upper left', fontsize=20)

plt.show()

In [ ]:
#df_consistent_smilesに含まれるペアデータ数、標的数、ユニーク化合物数、アッセイ数を表示
df_consistent_smiles = pd.read_csv('/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentsmiles.csv')
num_pairs = len(df_consistent_smiles)
num_targets = df_consistent_smiles['TID'].nunique()
num_unique_compounds = pd.unique(df_consistent_smiles[['REF-SMILES', 'PRB-SMILES']].values.ravel()).size
num_assays = df_consistent_smiles['AID'].nunique()
print(f"ペアデータ数: {num_pairs}")
print(f"標的数: {num_targets}")
print(f"ユニーク化合物数: {num_unique_compounds}")
print(f"アッセイ数: {num_assays}")

In [ ]:
# df_consistent_smilesのlabel_binごとの件数と%を表示
df_consistent_smiles = pd.read_csv('/gs/bs/tga-ohuelab/masunaga/MMP_dataset/dataset_consistentsmiles.csv')
label_bin_counts = df_consistent_smiles['label_bin'].value_counts().sort_index()
print("label_binごとの件数:")
print(label_bin_counts) 
label_bin_percent = df_consistent_smiles['label_bin'].value_counts(normalize=True).sort_index() * 100
print("\nlabel_binごとの割合 (%):")
print(label_bin_percent)